In [ ]:
import numpy as np
import scipy.sparse as ss
import matplotlib.pyplot as plt
import pandas as pd

import corextopic.corextopic as ct
import corextopic.vis_topic as vt # jupyter notebooks will complain matplotlib is being loaded twice

from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
news_data = pd.read_csv("Individual_Tweets_Final.csv")

In [ ]:
news_data.head()

,Datetime,Week,Tweet Id,Username,Like Count,Display Name,Text_lemmatized
0,2021-01-01,1,1.345140e+18,QuantData,0,Quant Data,"['astrazeneca', 'expects', 'supply', '2m', 'do..."
1,2021-01-01,1,1.345120e+18,bosfloss,3,Jackie Haddon,"['r_impiombato', 'people', 'make', 'living', '..."
2,2021-01-01,1,1.345120e+18,like_argue,0,Wet Ass Pword,"['america', 'starting', '2021', '227315', 'new..."
3,2021-01-01,1,1.345120e+18,AdmiralWilhelm,0,Johannes Wilhelm,"['cnn', 'u', 'surpasses', '20', 'million', 'co..."
4,2021-01-01,1,1.345120e+18,LockeLiberal1,1,Glorious Leader Locke,"['r_h_ebright', 'christosargyrop', 'polychotom..."


In [ ]:
news_data['Text_lemmatized1'] = news_data['Text_lemmatized'].str.replace('[^\w\s]','')

<ipython-input-4-8123d839feb6>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  news_data['Text_lemmatized1'] = news_data['Text_lemmatized'].str.replace('[^\w\s]','')


In [ ]:
#Removal of stop words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rashidesai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
new = ("covid19 vaccine", "covid19", "vaccine", "coronavirus", "vaccination", "covid", "corona"
      , "virus", "covid vaccine", "covidvaccine", "covid19vaccine", "im" ,  "wont" ,  "2021" ,  "21" ,  "2020" ,  "20" ,  
       "äì" ,  "dont" ,  "cant" ,  "via" ,  "already" ,  "al" ,  "úö" ,  "5g" ,  "covid_19" ,  "rt" ,  "ai" ,  "donäôt" ,  "iäôm" ,  "itäôs" ,   
       "dos" ,  "13" ,  "14" ,  "dy" ,  "amplitude" , "jampj", "pfizer" ,  "pfizers" ,  "az" ,  "astrazeneca" ,  "johnson" ,  "covax" ,  
       "moderna", "serum", "oxford", "covaxin", "covishield", "sinopharm", "novavax", "zeneca", "sputnik", "oxfordastrazeneca", 
       "letäôs", "covids", "pfizerbiontech", "doesnäô", "seruminstindia", "bharatbiotech", "biotech", "astrazenecas", "astrazenecaoxford", "covidー19")
stop.extend(new)

In [ ]:
news_data['Tokens']  = news_data['Text_lemmatized1'] .apply(lambda x: " ".join(x for x in x.split() if x not in stop))
news_data.head()

,Datetime,Week,Tweet Id,Username,Like Count,Display Name,Text_lemmatized,Text_lemmatized1,Tokens
0,2021-01-01,1,1.345140e+18,QuantData,0,Quant Data,"['astrazeneca', 'expects', 'supply', '2m', 'do...",astrazeneca expects supply 2m dos covid19 vacc...,expects supply 2m every week uk related ticker...
1,2021-01-01,1,1.345120e+18,bosfloss,3,Jackie Haddon,"['r_impiombato', 'people', 'make', 'living', '...",r_impiombato people make living tourism disadv...,r_impiombato people make living tourism disadv...
2,2021-01-01,1,1.345120e+18,like_argue,0,Wet Ass Pword,"['america', 'starting', '2021', '227315', 'new...",america starting 2021 227315 new covid case 34...,america starting 227315 new case 3419 new deat...
3,2021-01-01,1,1.345120e+18,AdmiralWilhelm,0,Johannes Wilhelm,"['cnn', 'u', 'surpasses', '20', 'million', 'co...",cnn u surpasses 20 million covid19 case expert...,cnn u surpasses million case expert warn itll ...
4,2021-01-01,1,1.345120e+18,LockeLiberal1,1,Glorious Leader Locke,"['r_h_ebright', 'christosargyrop', 'polychotom...",r_h_ebright christosargyrop polychotomy __ice9...,r_h_ebright christosargyrop polychotomy __ice9...


In [ ]:
news_data['Week'].value_counts()

12    267170
5     222731
11    203533
4     203272
10    196856
3     167140
1     165860
2     159618
9     151928
6     124551
8     123921
7     122847
Name: Week, dtype: int64

In [ ]:
news_data = news_data[news_data['Week']== 2]
news_data.shape

(159618, 9)

In [ ]:
# Transform data into a sparse matrix
vectorizer = CountVectorizer(stop_words='english', max_features=None, binary=True)
doc_word = vectorizer.fit_transform(news_data.Tokens)
doc_word = ss.csr_matrix(doc_word)

doc_word.shape # n_docs x m_words

(159618, 223817)

In [ ]:
doc_word

<159618x223817 sparse matrix of type '<class 'numpy.int64'>'
	with 1914412 stored elements in Compressed Sparse Row format>

In [ ]:
# Get words that label the columns (needed to extract readable topics and make anchoring easier)
words = list(np.asarray(vectorizer.get_feature_names()))

In [ ]:
anchor_words = [ 
#vaccine eligibility
['65', 'department', 'eligible', 'hospitalized', '75', 'local', 'resident', 'senior', '80', '95', 'old', 'elderly',
  '21', 'age', 'eligibility'], 
#registration 
 ['booked', 'book', 'schedule', 'call', 'website', 'email', 'message', 'letter', 'phone', 'walk', 
 'appointment', 'scheduling', 'slots', 'availability', 'available', 'open', 'apply', 'demand', 'register', 
 'registration', 'pre-register', 'portal', 'line', 'hotline', 'same-day', 'next-day', 'online', 'waiting', 'list', 
 'sign-up', 'contact', 'drive-thru', 'no vaccine appointments', 'reschedule', 'rejected', 'cancel', 'maxed', 
 'no-appointment', 'hurdles', 'impossible', 'cancellation', 'no response', 'obstacles', 'scheduled'], 
#vaccine approval
 ['approved', 'approval', 'clinical', 'trial', 'approves', 'rollout', 'speed', 'plan', 'release', 'distribution', 
  'holding', 'million', 'billion', 'official', 'delivered', 'production', 'supply', 'reported', 'petition', 'rallied',
  'effective', 'study', 'impeachment', 'experiment', 'export', 'testing', 'phase', 'phases'], 
#personas
 ['boris',  'johnson', 'narendra', 'modi', 'trump', 'biden', 'justin', 'trudeau',  'joe', 'namo', 'janet yellen', 
  'ron', 'desantis', 'von der leyen', 'realdonaldtrump', 'bruno le maire', 'paolo gentiloni', 'olaf scholz', 'cuomo', 
  'fauci'], 
#travel
 ['passport', 'access', 'travel', 'risk', 'airline', 'plane', 'airfare', 'country', 'visa', 'flight', 'train', 
  'airport', 'trip', 'hotel', 'vacation', 'ban', 'border', 'sealed', 'fly', 'vaccinepassports'],
#pandemic
['variant', 'strains', 'mutants', 'transmission', 'zone', 'red', 'green', 'wuhan', 'containment', 'spread', 
 'pandemic', 'epidemic', 'un', 'who', 'curve', 'herd immunity', 'mask', '6', 'feet', 'social', 'distancing', 
 'maintain', 'isolation', 'quarantine', '7', 'shield', 'ppe', 'oxygen', 'ventilator', 'emergency', 'capacity', 
 'occupancy', 'confirmed', 'global', 'worldwide', 'hydroxychloroquine', 'sars', 'cov', 'endemic', 'outbreak', 
 'cluster', 'lockdown', 'sanitized', 'sanitizer', 'prevent', 'catch', 'spreading', 'catching', 'passing', 'avoid', 
 'crowd', 'gathering', 'hand', 'mutation', 'safety', 'new', 'n-95', 'n95', 'test', 'guideline', 'reopening', 'cdc'],
#geography, politics
 ['geography', 'usa' ,'uk', 'india', 'eu', 'brexit', 'scotland', 'france', 'canada', 'china', 'africa', 'somalia', 
  'brazil', 'america', 'britain', 'israel', 'germany', 'denmark', 'nigerian', 'portugese', 'mexican', 'israeli', 
  'indian', 'taiwan', 'malaysia', 'cuba', 'canadian', 'mexico', 'latino', 'vatican', 'british', 'ministry', 
  'president', 'congress', 'republic', 'prime', 'minister', 'potus', 'administrator', 'administration', 
  'governor', 'cnn', 'government', 'bbc', 'prince' , 'news', 'scam', 'misinformation', 'information', 'text', 
  'info', 'facebook', 'fake', 'data', 'detail', 'link', 'page', 'instagram', 'medium', 'media', 'tv', 'video', 
  'audio',  'youtube'],
#healthcare workers
 ['healthcare', 'workers', 'doctor', 'dr', 'nurse', 'employees', 'student', 'frontline', 'essential',  'worker', 
  'volunteer', 'teacher', 'staff', 'patient', 'care', 'school', 'offered', 'nursing', 'working', 'people', 
  'medical', 'plant', 'home', 'military', 'team', 'employee', 'employer', 'scientist', 'physician', 'family',  
  'member', 'providers', 'healthcare'],
#people for vaccine
 ['getvaccinated', 'readytovaccinate', 'goandgetvaccinate', 'vaccinated', 'vaccineswork', 'vaccinessavelives', 
  'get', 'antibody', 'saves', 'ready', 'go', 'goandget', 'work', 'save', 'saves'], 
# people against vaccine
  ['antivaccine', 'vaccinekills', 'novaccination', 'stopvaccines', 'nomandatoryvaccine', 'forced vaccination', 
   'justsayno', 'nogoodvaccine', 'novaccineforme', 'boycottvaccine', 'sideeffects', 'saynotovaccines', 'chip', 
  'danger', 'micro', 'risky', 'stop', 'warning', 'warn', 'anti', 'kills', 'no', 'mandatory', 'sayno', 'nogood', 
  'novaccine', 'boycott'],
#who got vaccinated
 ['mother', 'father', 'self', 'mom', 'woman', 'granny', 'dad', 'mum', 'child', 'son', 'daughter', 'grandpa', 
  'kid', 'husband', 'wife', 'spouse', 'grandparent', 'parent', 'grandma', 'uncle', 'aunt'], 
#vaccination status
  ['yesterday', 'morning', 'today', 'feb', 'jan', 'march', 'january', 'february', 'week', 'day', 'tomorrow', 'year', 'hour', 
  'ago', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 'weekend', 'weekday', 
  'last', 'got', 'gotten', 'received', 'dose-1', 'dose-2', 'dose 1', 'one down', 'dose 2', 'first', 'second', 
  'finally'],
#sideeffects
 ['symptom', 'fever', 'chills', 'body', 'pain', 'paralysis', 'death', 'fatigue', 'headache', 'nausea', 
  'vomiting', 'muscle', 'sore', 'swollen', 'blood', 'clot', 'sleepy', 'sickness', 'malaise', 'clots', 'tired',
  'tiredness', 'dizziness', 'numbness', 'cardiac', 'arrest', 'chest', 'cough', 'allergies', 'side effects',  
  'vaccinesideeffects', 'jab', 'positive', 'shot', 'adverse', 'infection', 'symptoms', 'ill', 
  'depression', 'died', 'reaction', 'asthma', 'pressure', 'illness', 'allergy', 'pregnant', 'allergic', 'fertility', 
  'anaphylaxis', 'asthmatic', 'inject', 'injection',  'happy', 'good', 'excited', 'grateful', 'worry', 'scared', 
  'worried', 'relief', 'vulnerable', 'ache'], 
#vaccination sites
 ['town', 'hall', 'state', 'county', 'city', 'location', 'miles', 'radius', 'distance', 'drive', 'zip', 'pin', 
  'code', 'drove', 'near', 'within', 'gas', 'away', 'far', '10', '50', 'hours', 'site', 'clinic', 'pharmacy', 
  'centers', 'walgreens', 'hospital', 'church', 'temple', 'community', 'communities', 'campus', 'regional', 
  'mosque', 'baptist', 'sites', 'mass', 'black', 'asian', 'hispanic']]

anchored_topic_model = ct.Corex(n_hidden=14, seed=2)
anchored_topic_model.fit(doc_word, words=words, anchors=anchor_words, anchor_strength=2);

In [ ]:
for n in range(len(anchor_words)):
    topic_words,_,_ = zip(*anchored_topic_model.get_topics(topic=n))
    print('{}: '.format(n) + ', '.join(topic_words))

0: 65, resident, old, eligible, age, local, senior, elderly, department, 75
1: available, appointment, line, waiting, open, list, schedule, online, website, register
2: rollout, distribution, million, plan, trial, phase, supply, testing, official, speed
3: trump, biden, modi, fauci, joe, cuomo, desantis, trudeau, election, namo
4: country, risk, access, travel, passport, ban, iran, border, airport, flight
5: new, pandemic, mask, test, spread, variant, lockdown, social, safety, prevent
6: government, uk, india, china, israel, president, canada, minister, africa, news
7: worker, care, home, patient, staff, healthcare, dr, medical, working, family
8: work, vaccinated, ready, save, antibody, herd, immunity, vaccineswork, vaccinessavelives, getvaccinated
9: stop, mandatory, warning, anti, chip, obligation, danger, occupation, power, warn
10: mom, woman, child, parent, dad, kid, mother, mum, wife, husband
11: got, today, day, week, second, received, dose, hour, tomorrow, yesterday
12: death,

In [ ]:
vt.vis_rep(anchored_topic_model, column_label=words, prefix='weektwo')

Print topics in text file
